###### Save this notebook to the `Notebooks` directory.<br><br>
The code follows the structure of the Wind repository on big Andy's Github. In theory, if you have cloned the repository, the code should handle data retrieval when the notebook is saved in the `Notebooks` directory.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as tts
from sklearn.grid_search import GridSearchCV

import matplotlib.pyplot as plt
import copy

%matplotlib inline

C:\Users\Dave Landay\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Dave Landay\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Build the data:

In [3]:
#Test to see that we are backing out of `Notebooks` and retrieving data from Wind parent directory:
path = os.getcwd()
print('current: '+ path)
os.chdir('..')
path = os.getcwd()
print('back one dir: '+path+'\n')

# Define path extension where data lives and the file we want to open:
path2data = '\CS295\procData\ourData\FinalData'
file_of_interest = '2015Avg.txt'

#load Data into a dataframe (define the header names appropriately):
data = pd.read_csv(path+path2data+'\\'+file_of_interest, names=['Lat','Lon','Ele','YR--MODAHRMN','DIR','SPD','GUS',
                                                                  'TEMP','SLP'])

current: C:\Users\Dave Landay\Documents\Machine Learning\GroupProject\wind\Notebooks
back one dir: C:\Users\Dave Landay\Documents\Machine Learning\GroupProject\wind



In [4]:
#delete the gust column (to many Null values):
del data['GUS']

#Display first ten entries:
data[:10]

,Lat,Lon,Ele,YR--MODAHRMN,DIR,SPD,TEMP,SLP
0,60.383,22.55,5.2,20150101,4.712389,2.840804,35.75,1007.03181818
1,60.383,22.55,5.2,20150102,4.712389,5.648369,38.5,981.977272727
2,60.383,22.55,5.2,20150103,-1.516701,4.951222,35.6818181818,978.972727273
3,60.383,22.55,5.2,20150104,0.495934,6.277769,29.8636363636,997.059090909
4,60.383,22.55,5.2,20150105,0.379936,6.052150,13.9166666667,1020.12083333
5,60.383,22.55,5.2,20150106,1.570796,1.673521,11.75,1030.37083333
6,60.383,22.55,5.2,20150107,4.712389,7.661832,32.4583333333,1019.10416667
7,60.383,22.55,5.2,20150108,4.712389,3.511538,35.625,1002.35
8,60.383,22.55,5.2,20150109,4.712389,1.809526,33.2083333333,989.120833333
9,60.383,22.55,5.2,20150110,1.570796,1.609291,31.3333333333,982.275


In [5]:
#Descritize Wind Speed (Use Beaufort Wind Scale):
def beaufort(value):
    """BEAUFORT takes a wind speed value (in m/s) and classifies  it based on the Beaufort Wind Scale:
    0 --- Calm ----------- less than 1 mph (0 m/s)
    1 --- Light air ------ 1 - 3 mph 0.5-1.5 m/s
    2 --- Light breeze --- 4 - 7 mph 2-3 m/s
    3 --- Gentle breeze -- 8 - 12 mph 3.5-5 m/s
    4 --- Moderate breeze  13 - 18 mph 5.5-8 m/s
    5 --- Fresh breeze --- 19 - 24 mph 8.5-10.5 m/s
    6 --- Strong breeze -- 25 - 31 mph 11-13.5 m/s
    7 --- Moderate gale -- 32 - 38 mph 14-16.5 m/s
    8 --- Fresh gale  ---- 39 - 46 mph 17-20 m/s
    9 --- Strong gale ---- 47 - 54 mph 20.5-23.5 m/s
    10 -- Whole gale ----- 55 - 63 mph 24-27.5 m/s
    11 -- Storm ---------- 64 - 73 mph 28-31.5 m/s
    12 -- Hurricane ------ over 73 mph over 32 m/s"""
    
    #Define intervals:
    if value < 0.5:
        value = 0
    if value >= 0.5 and value < 2:
        value = 1
    if value >= 2 and value < 3:
        value = 2
    if value >=3 and value < 5:
        value = 3
    if value >=5 and value < 8:
        value = 4
    if  value >=8 and value < 10.5:
        value = 5
    if value >=10.5 and value < 13.5:
        value = 6
    if value >= 13.5 and value < 16.5:
        value = 7
    if value >= 16.5 and value < 20:
        value = 8
    if value >= 20 and value < 23.5:
        value = 9
    if value >= 23.5 and value < 27.5:
        value = 10
    if value >= 27.5 and value < 31.5:
        value = 11
    if value >= 31.5:
        value = 12
        
    return value

In [6]:
#Apply the transformation to 'SPD' Column:
data['SPD'] = data['SPD'].map(beaufort)

#sanity check:
data[:10]
    

,Lat,Lon,Ele,YR--MODAHRMN,DIR,SPD,TEMP,SLP
0,60.383,22.55,5.2,20150101,4.712389,2,35.75,1007.03181818
1,60.383,22.55,5.2,20150102,4.712389,4,38.5,981.977272727
2,60.383,22.55,5.2,20150103,-1.516701,3,35.6818181818,978.972727273
3,60.383,22.55,5.2,20150104,0.495934,4,29.8636363636,997.059090909
4,60.383,22.55,5.2,20150105,0.379936,4,13.9166666667,1020.12083333
5,60.383,22.55,5.2,20150106,1.570796,1,11.75,1030.37083333
6,60.383,22.55,5.2,20150107,4.712389,4,32.4583333333,1019.10416667
7,60.383,22.55,5.2,20150108,4.712389,3,35.625,1002.35
8,60.383,22.55,5.2,20150109,4.712389,1,33.2083333333,989.120833333
9,60.383,22.55,5.2,20150110,1.570796,1,31.3333333333,982.275


In [7]:
# fix the DIR interval to be [0,2pi]
def fixInterval(values):
    #Add 2pi to any negative value found in the DIR column:
    if values < 0:
        values+=2*np.pi
    return values

In [8]:
#Transform 'DIR'  column:
data['DIR'] = data['DIR'].map(fixInterval)

#sanity check:
data['DIR'].describe()

count    20752.000000
mean         3.875966
std          1.672421
min          0.000228
25%          1.570796
50%          4.712389
75%          4.712389
max          6.283185
Name: DIR, dtype: float64

In [9]:
#Make Cardinal Directions:
def makeDirection(values):
    """MAKEDIRECTION puts all DIR values on the closed interval [0,2pi]. 8 Subdivisions will define ranges for 
    the 8 cardinal directions. If a value is an element of one of the 8 sub-intervals, then it will be encoded
    as follows:
    0--N: ------[0,pi/8]U[15pi/8,2pi]
    1--NE:------[pi/8,3pi/8]
    2--E: ------[3pi/8,5pi/8]
    3--SE:------[5pi/8,7pi/8]
    4--S: ------[7pi/8,9pi/8] 
    5--SW:------[9pi/8,11pi/8]
    6--W: ------[11pi/8,13pi/8]
    7--NW:------[13pi/8,15pi/8]"""
    
    #encode values:
    if (values >= 0 and values <= np.pi/8) or (values >= 15*np.pi/8 and values <= 2*np.pi):
        values = 0
    elif values > np.pi/8 and values < 3*np.pi/8:
        values = 1
    elif values >= 3*np.pi/8 and values <= 5*np.pi/8:
        values = 2
    elif values > 5*np.pi/8 and values < 7*np.pi/8:
        values = 3
    elif values >= 7*np.pi/8 and values <= 9*np.pi/8:
        values = 4
    elif values > 9*np.pi/8 and values < 11*np.pi/8:
        values = 5
    elif values >= 11*np.pi/8 and values <= 13*np.pi/8:
        values = 6
    elif values > 13*np.pi/8 and values < 15*np.pi/8:
        values = 7
    return values

In [10]:
#Transform 'DIR' column:
data['DIR'] =data['DIR'].map(makeDirection)

#Sanity check:
data[:10]

,Lat,Lon,Ele,YR--MODAHRMN,DIR,SPD,TEMP,SLP
0,60.383,22.55,5.2,20150101,6,2,35.75,1007.03181818
1,60.383,22.55,5.2,20150102,6,4,38.5,981.977272727
2,60.383,22.55,5.2,20150103,6,3,35.6818181818,978.972727273
3,60.383,22.55,5.2,20150104,1,4,29.8636363636,997.059090909
4,60.383,22.55,5.2,20150105,0,4,13.9166666667,1020.12083333
5,60.383,22.55,5.2,20150106,2,1,11.75,1030.37083333
6,60.383,22.55,5.2,20150107,6,4,32.4583333333,1019.10416667
7,60.383,22.55,5.2,20150108,6,3,35.625,1002.35
8,60.383,22.55,5.2,20150109,6,1,33.2083333333,989.120833333
9,60.383,22.55,5.2,20150110,2,1,31.3333333333,982.275


In [11]:
for val in data['TEMP']:
    print(type(val))
    break

<class 'str'>


In [12]:
for val in data['SLP']:
    print(type(val))
    break

<class 'str'>


That's not good! 

In [13]:
#Convert 'TEMP' and 'SLP' columns into floats:
def convert2Float(values):
    """CONVERT2FLOAT takes a string argument and converts it to a float type"""
    return float(values)

In [14]:
#Convert to floats:
data['SLP'] = data['SLP'].map(convert2Float)
data['TEMP'] = data['TEMP'].map(convert2Float)

#Sanity check:
for val in data['SLP']:
    print(type(val))
    break
    
for val in data['TEMP']:
    print(type(val))
    break

ValueError: could not convert string to float: '***'

This means we have a string that is not a number:

In [15]:
count = 0
index_2_remove = []
for idx,val in enumerate(data['TEMP']):
    if '*' in val:
        count+=1
        index_2_remove.append(idx)
        print(idx,val)
print(count)  

12270 ***
12273 ***
2


In [16]:
count = 0

for idx,val in enumerate(data['SLP']):
    if '*' in val:
        count+=1
        index_2_remove.append(idx)
print(count)  

3662


In [17]:
#get a sense of how much the data will shrink by:
data.shape

(20752, 8)

In [18]:
#prepare indices for iterative removal:
index_2_remove = set(index_2_remove)

#sanity check:
len(index_2_remove)

3662

Drop the rows at each index in `index_2_remove`:

In [19]:
for idx in index_2_remove:
    data.drop(idx, inplace=True)

#Sanity check:
data.shape

(17090, 8)

In [20]:
#Convert to floats:
data['SLP'] = data['SLP'].map(convert2Float)
data['TEMP'] = data['TEMP'].map(convert2Float)

In [21]:
#Sanity check:
for val in data['SLP']:
    print(type(val))
    break
    
for val in data['TEMP']:
    print(type(val))
    break

<class 'numpy.float64'>
<class 'numpy.float64'>


In [22]:
#check the range of temperatures:
data['TEMP'].describe()

count    17090.000000
mean        44.316354
std         12.164029
min         -8.250000
25%         35.272727
50%         43.028523
75%         54.909091
max         74.545455
Name: TEMP, dtype: float64

In [23]:
#check the range of pressures:
data['SLP'].describe()

count    17090.000000
mean      1011.449581
std         12.840634
min        966.559091
25%       1003.666667
50%       1011.370833
75%       1018.803125
max       1053.604167
Name: SLP, dtype: float64

Will this amount of variability affect predictive power?

Because wind speed and direction are correlated, this becomes a multi-output problem where the target is a 2D array of size 2 x 17090.<br><br>
This feature is supported inherantly in sklearn's RandomForestClassifier() module... DOPE!

In [24]:
#prepare training and testing data. partition 80% train, 20% testing:
X = pd.concat([data['Lat'],data['Lon'],data['Ele'],data['YR--MODAHRMN'],data['TEMP'],data['SLP']], axis=1)
y = pd.concat([data['SPD'],data['DIR']], axis=1)

#sanity check:
print(X[:10])
print(y[:10])

      Lat    Lon  Ele  YR--MODAHRMN       TEMP          SLP
0  60.383  22.55  5.2      20150101  35.750000  1007.031818
1  60.383  22.55  5.2      20150102  38.500000   981.977273
2  60.383  22.55  5.2      20150103  35.681818   978.972727
3  60.383  22.55  5.2      20150104  29.863636   997.059091
4  60.383  22.55  5.2      20150105  13.916667  1020.120833
5  60.383  22.55  5.2      20150106  11.750000  1030.370833
6  60.383  22.55  5.2      20150107  32.458333  1019.104167
7  60.383  22.55  5.2      20150108  35.625000  1002.350000
8  60.383  22.55  5.2      20150109  33.208333   989.120833
9  60.383  22.55  5.2      20150110  31.333333   982.275000
   SPD  DIR
0    2    6
1    4    6
2    3    6
3    4    1
4    4    0
5    1    2
6    4    6
7    3    6
8    1    6
9    1    2


In [25]:
#split the data:
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.2, random_state = 0)

#sanity check:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


(13672, 6) (3418, 6) (13672, 2) (3418, 2)


How many Decision trees should be used to build the random forest?

In [26]:
#Create the model:
forest = RandomForestClassifier(n_estimators=100, random_state=0)
multi_output_forest = MultiOutputClassifier(forest, n_jobs = -2) #Use all but 1 CPU's

#Fit the model:
multi_output_forest.fit(X_train, y_train).predict(X_train)

array([[4, 6],
       [4, 6],
       [5, 2],
       ..., 
       [5, 1],
       [4, 6],
       [4, 6]], dtype=int64)

Check the accuracy:

In [27]:
scores = cross_val_score(multi_output_forest, X_train, y_train)
print(scores)

[ 0.2064502   0.20847048  0.22163704]


Hmmm... Pretty shitty